In [1]:
# class Encoder(nn.Module):
#     def __init__(self, in_channels, hidden_channels, num_layers, heads, dropout=0.1):
#         super(Encoder, self).__init__()
#         self.num_layers = num_layers
#         self.convs = nn.ModuleList()
        
#         # 添加多层 GATConv
#         self.convs.append(GATConv(in_channels, hidden_channels, heads=heads))
#         for _ in range(num_layers - 1):
#             self.convs.append(GATConv(hidden_channels * heads, hidden_channels, heads=heads))
        
#         # 根据隐藏特征的维度设置 PReLU 参数数量
#         self.prelu = nn.PReLU(num_parameters=hidden_channels)

#         # 添加 Dropout 层
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, edge_index):
#         # 定义一个变量用于保存输入特征，以便进行跳跃连接
#         residual = x
        
#         for conv in self.convs:
#             # 对输入特征进行图注意力操作
#             x = conv(x, edge_index)
            
#             # 将图注意力操作的结果与原始输入特征相加
#             x = x + residual

#             # 应用 Dropout 正则化
#             x = self.dropout(x)
            
#             # 应用 PReLU 激活函数
#             x = self.prelu(x)
            
#             # 更新跳跃连接的输入特征为当前层的输出
#             residual = x
        
#         return x

### Prepare data

In [66]:
import pandas as pd
import pickle
import numpy as np

In [3]:
# 读取原始数据（假设每个诊断记录为一行）
clinical_df = pd.read_csv("/home/llma/wzy/comorbidity/Data/ICD10_long.csv")

In [4]:
clinical_df.shape

(7015554, 5)

In [5]:
clinical_df.head(2)

,eid,ICD10_codes,Date,p40000_i0,p40023
0,1000122,J383,1999-07-06,2014-12-22,1.0
1,1000122,M255,2001-07-31,2014-12-22,1.0


In [6]:
len(clinical_df['ICD10_codes'].unique())

12315

In [7]:
Olink_df = pd.read_csv("/home/llma/wzy/comorbidity/Data/Olink.csv")

In [8]:
Olink_df.head(2)

,olink_instance_0.eid,olink_instance_0.a1bg,olink_instance_0.aamdc,olink_instance_0.aarsd1,olink_instance_0.abca2,olink_instance_0.abhd14b,olink_instance_0.abl1,olink_instance_0.abo,olink_instance_0.abraxas2,olink_instance_0.acaa1,...,olink_instance_0.zfyve19,olink_instance_0.zhx2,olink_instance_0.znf174,olink_instance_0.znf75d,olink_instance_0.znf830,olink_instance_0.znrd2,olink_instance_0.znrf4,olink_instance_0.zp3,olink_instance_0.zp4,olink_instance_0.zpr1
0,1002133,0.0957,0.59795,1.33215,-0.6441,0.76325,0.3784,0.5446,0.4682,0.21920,...,-0.2728,0.1954,-0.2586,-0.0127,0.0235,-0.26245,-0.1236,-6.0773,-0.0136,0.4131
1,1002201,-0.0473,0.39585,0.26585,0.0604,1.07765,1.3977,0.5830,1.0740,0.75625,...,0.9153,-0.0274,-0.3048,-0.8814,-0.5342,1.12255,-0.5537,-5.1443,-0.1713,-0.5605


In [9]:
Olink_df = Olink_df.rename(columns={'olink_instance_0.eid': 'eid'})

In [10]:
Olink_df.head(2)

,eid,olink_instance_0.a1bg,olink_instance_0.aamdc,olink_instance_0.aarsd1,olink_instance_0.abca2,olink_instance_0.abhd14b,olink_instance_0.abl1,olink_instance_0.abo,olink_instance_0.abraxas2,olink_instance_0.acaa1,...,olink_instance_0.zfyve19,olink_instance_0.zhx2,olink_instance_0.znf174,olink_instance_0.znf75d,olink_instance_0.znf830,olink_instance_0.znrd2,olink_instance_0.znrf4,olink_instance_0.zp3,olink_instance_0.zp4,olink_instance_0.zpr1
0,1002133,0.0957,0.59795,1.33215,-0.6441,0.76325,0.3784,0.5446,0.4682,0.21920,...,-0.2728,0.1954,-0.2586,-0.0127,0.0235,-0.26245,-0.1236,-6.0773,-0.0136,0.4131
1,1002201,-0.0473,0.39585,0.26585,0.0604,1.07765,1.3977,0.5830,1.0740,0.75625,...,0.9153,-0.0274,-0.3048,-0.8814,-0.5342,1.12255,-0.5537,-5.1443,-0.1713,-0.5605


In [11]:
Olink_df.shape

(53013, 2924)

In [12]:
# 将数据框从宽格式转换为长格式
Olink_long_df = pd.melt(Olink_df, id_vars=['eid'], var_name='protein_name', value_name='protein_expression')

print(Olink_long_df)  

               eid           protein_name  protein_expression
0          1002133  olink_instance_0.a1bg              0.0957
1          1002201  olink_instance_0.a1bg             -0.0473
2          1002534  olink_instance_0.a1bg             -0.2239
3          1003548  olink_instance_0.a1bg              0.0255
4          1003599  olink_instance_0.a1bg              0.2003
...            ...                    ...                 ...
154956994  6021692  olink_instance_0.zpr1              1.2116
154956995  6021914  olink_instance_0.zpr1             -0.0460
154956996  6022081  olink_instance_0.zpr1             -0.6499
154956997  6022213  olink_instance_0.zpr1             -0.1758
154956998  6022247  olink_instance_0.zpr1              1.4512

[154956999 rows x 3 columns]


In [13]:
53013 * 2923

154956999

In [14]:
Olink_long_df = Olink_long_df.dropna(subset=['protein_expression'])

In [15]:
Olink_long_df.shape

(138994642, 3)

In [16]:
NMR_df = pd.read_csv("/home/llma/wzy/comorbidity/Data/NMR.csv")

In [17]:
NMR_df.head(2)

,participant.eid,participant.p20280_i0,participant.p20281_i0,participant.p23400_i0,participant.p23401_i0,participant.p23402_i0,participant.p23403_i0,participant.p23404_i0,participant.p23405_i0,participant.p23406_i0,...,participant.p23639_i0,participant.p23640_i0,participant.p23641_i0,participant.p23642_i0,participant.p23643_i0,participant.p23644_i0,participant.p23645_i0,participant.p23646_i0,participant.p23647_i0,participant.p23648_i0
0,1000053,2.9721,0.282259,3.8459,2.9413,1.5075,0.83349,1.9998,1.4338,0.90459,...,49.941,40.646,33.031,7.6145,9.413,58.373,34.427,24.479,9.9478,7.2007
1,1000122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
NMR_df = NMR_df.rename(columns={'participant.eid': 'eid'})

In [19]:
NMR_df.head(2)

,eid,participant.p20280_i0,participant.p20281_i0,participant.p23400_i0,participant.p23401_i0,participant.p23402_i0,participant.p23403_i0,participant.p23404_i0,participant.p23405_i0,participant.p23406_i0,...,participant.p23639_i0,participant.p23640_i0,participant.p23641_i0,participant.p23642_i0,participant.p23643_i0,participant.p23644_i0,participant.p23645_i0,participant.p23646_i0,participant.p23647_i0,participant.p23648_i0
0,1000053,2.9721,0.282259,3.8459,2.9413,1.5075,0.83349,1.9998,1.4338,0.90459,...,49.941,40.646,33.031,7.6145,9.413,58.373,34.427,24.479,9.9478,7.2007
1,1000122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# 将数据框从宽格式转换为长格式
NMR_long_df = pd.melt(NMR_df, id_vars=['eid'], var_name='NMR_name', value_name='NMR_expression')

print(NMR_long_df)  

               eid               NMR_name  NMR_expression
0          1000053  participant.p20280_i0         2.97210
1          1000122  participant.p20280_i0             NaN
2          1000148  participant.p20280_i0         5.79763
3          1000150  participant.p20280_i0         5.32703
4          1000163  participant.p20280_i0             NaN
...            ...                    ...             ...
126034876  6022213  participant.p23648_i0             NaN
126034877  6022230  participant.p23648_i0             NaN
126034878  6022247  participant.p23648_i0             NaN
126034879  6022252  participant.p23648_i0         5.26830
126034880  6022444  participant.p23648_i0             NaN

[126034881 rows x 3 columns]


In [21]:
NMR_df.shape

(502131, 252)

In [22]:
502131 * 251

126034881

In [23]:
# 移除 NMR_expression 列中值为 NaN 的行
NMR_long_df = NMR_long_df.dropna(subset=['NMR_expression'])

print(NMR_long_df) 

               eid               NMR_name  NMR_expression
0          1000053  participant.p20280_i0         2.97210
2          1000148  participant.p20280_i0         5.79763
3          1000150  participant.p20280_i0         5.32703
5          1000199  participant.p20280_i0         5.73236
8          1000428  participant.p20280_i0         5.26226
...            ...                    ...             ...
126034871  6022081  participant.p23648_i0         5.61880
126034872  6022102  participant.p23648_i0         5.26160
126034874  6022141  participant.p23648_i0         4.78620
126034875  6022197  participant.p23648_i0         3.85360
126034879  6022252  participant.p23648_i0         5.26830

[68736894 rows x 3 columns]


In [24]:
# 筛选出 clinical_df 中 eid 也存在于 Olink_df 的 eid 列中的行
subset_clinical_df = clinical_df[clinical_df['eid'].isin(Olink_df['eid'])]

In [25]:
subset_clinical_df.shape

(805376, 5)

In [26]:
# 筛选出 clinical_df 中 eid 也存在于 Olink_df 的 eid 列中的行
subset_clinical_df = subset_clinical_df[subset_clinical_df['eid'].isin(NMR_df['eid'])]

In [27]:
subset_clinical_df.shape

(805376, 5)

In [28]:
subset_clinical_df.head(2)

,eid,ICD10_codes,Date,p40000_i0,p40023
743,1002133,I802,2003-04-02,NaN,NaN
744,1002133,M1997,2009-04-17,NaN,NaN


In [29]:
len(subset_clinical_df['eid'].unique())

47626

In [30]:
Olink_df['eid'].isin(clinical_df['eid']).sum()

47626

In [31]:
Olink_df['eid'].isin(subset_clinical_df['eid']).sum()

47626

In [32]:
# 保存为以制表符分隔的文本文件
file_path = '/home/llma/wzy/comorbidity/Data/subset_ICD10_in_Olink_long.txt'
subset_clinical_df.to_csv(file_path, sep='\t', na_rep='nan', index=False)

In [33]:
subset_clinical_df.head(2)

,eid,ICD10_codes,Date,p40000_i0,p40023
743,1002133,I802,2003-04-02,NaN,NaN
744,1002133,M1997,2009-04-17,NaN,NaN


In [34]:
subset_clinical_df = subset_clinical_df.drop(columns=['Date', 'p40000_i0', 'p40023'])

In [35]:
subset_clinical_df['relationship'] = 1

In [36]:
subset_clinical_df.head(2)

,eid,ICD10_codes,relationship
743,1002133,I802,1
744,1002133,M1997,1


In [37]:
subset_Olink_long_df = Olink_long_df[Olink_long_df['eid'].isin(subset_clinical_df['eid'])]

In [38]:
subset_Olink_long_df.head(2)

,eid,protein_name,protein_expression
0,1002133,olink_instance_0.a1bg,0.0957
1,1002201,olink_instance_0.a1bg,-0.0473


In [39]:
subset_Olink_long_df.shape

(124814467, 3)

In [40]:
# 按 eid 分组并统计每组的记录数量
eid_counts = subset_Olink_long_df.groupby('eid').size()

# 找出记录数量少于 200 的 eid
eid_less_than_200 = eid_counts[eid_counts < 200].index.tolist()

print("记录数量少于 200 的 eid:", eid_less_than_200)

记录数量少于 200 的 eid: [1949202, 2292307, 3139172, 3783685, 4156288]


In [41]:
# 按 eid 分组，对每个分组内按 protein_expression 降序排序
sorted_Olink_df = subset_Olink_long_df.sort_values(['eid', 'protein_expression'], ascending=[True, False])

# 对每个分组保留前 200 行
result_Olink_df = sorted_Olink_df.groupby('eid').head(200)

print(result_Olink_df)

               eid             protein_name  protein_expression
19805682   1000021   olink_instance_0.ccar2             9.35690
103884300  1000021    olink_instance_0.paep             4.74010
121378590  1000021    olink_instance_0.rln2             4.67330
88775595   1000021  olink_instance_0.mapre3             4.55670
54476184   1000021    olink_instance_0.fdx2             3.96610
...            ...                      ...                 ...
123159717  6022412   olink_instance_0.rtn4r             0.70720
805713     6022412  olink_instance_0.acot13             0.70610
134663538  6022412  olink_instance_0.spink6             0.70325
44488425   6022412   olink_instance_0.dnpep             0.70310
147068580  6022412    olink_instance_0.treh             0.70270

[9524997 rows x 3 columns]


In [42]:
result_Olink_df.shape

(9524997, 3)

In [43]:
len(result_Olink_df['eid'].unique())

47626

In [44]:
47626 * 200 - 9524997

203

In [45]:
subset_NMR_long_df = NMR_long_df[NMR_long_df['eid'].isin(subset_clinical_df['eid'])]

In [46]:
subset_NMR_long_df.shape

(6841649, 3)

In [47]:
subset_NMR_long_df.head(2)

,eid,NMR_name,NMR_expression
44,1002133,participant.p20280_i0,4.06436
47,1002201,participant.p20280_i0,5.62458


In [48]:
# 按 eid 分组并统计每组的记录数量
eid_counts = subset_NMR_long_df.groupby('eid').size()

# 找出记录数量少于 200 的 eid
eid_less_than_200 = eid_counts[eid_counts < 200].index.tolist()

print("记录数量少于 200 的 eid:", eid_less_than_200)

记录数量少于 200 的 eid: []


In [49]:
# 按 eid 分组，对每个分组内按 NMR_expression 降序排序
sorted_NMR_df = subset_NMR_long_df.sort_values(['eid', 'NMR_expression'], ascending=[True, False])

# 对每个分组保留前 200 行
result_NMR_df = sorted_NMR_df.groupby('eid').head(200)

print(result_NMR_df)

               eid               NMR_name  NMR_expression
93195467   1000021  participant.p23583_i0       87.447000
109263659  1000021  participant.p23615_i0       73.625000
98216777   1000021  participant.p23593_i0       73.483000
106753004  1000021  participant.p23610_i0       71.083000
111774314  1000021  participant.p23620_i0       68.700000
...            ...                    ...             ...
54832534   6022412  participant.p23507_i0        0.092613
54330403   6022412  participant.p23506_i0        0.086871
46798438   6022412  participant.p23491_i0        0.085022
37257949   6022412  participant.p23472_i0        0.085019
62364499   6022412  participant.p23522_i0        0.085007

[5459000 rows x 3 columns]


In [50]:
len(subset_NMR_long_df['eid'].unique())

27295

In [51]:
27295*200

5459000

In [52]:
subset_clinical_df = subset_clinical_df.rename(columns={'eid': 'node1'})
subset_clinical_df = subset_clinical_df.rename(columns={'ICD10_codes': 'node2'})

# subset_Olink_long_df = subset_Olink_long_df.rename(columns={'eid': 'node1'})
# subset_Olink_long_df = subset_Olink_long_df.rename(columns={'protein_name': 'node2'})
# subset_Olink_long_df = subset_Olink_long_df.rename(columns={'protein_expression': 'relationship'})

# subset_NMR_long_df = subset_NMR_long_df.rename(columns={'eid': 'node1'})
# subset_NMR_long_df = subset_NMR_long_df.rename(columns={'NMR_name': 'node2'})
# subset_NMR_long_df = subset_NMR_long_df.rename(columns={'NMR_expression': 'relationship'})

result_Olink_df = result_Olink_df.rename(columns={'eid': 'node1'})
result_Olink_df = result_Olink_df.rename(columns={'protein_name': 'node2'})
result_Olink_df = result_Olink_df.rename(columns={'protein_expression': 'relationship'})

result_NMR_df = result_NMR_df.rename(columns={'eid': 'node1'})
result_NMR_df = result_NMR_df.rename(columns={'NMR_name': 'node2'})
result_NMR_df = result_NMR_df.rename(columns={'NMR_expression': 'relationship'})

In [53]:
# 合并三个数据框
# merged_df = pd.concat([subset_clinical_df, subset_Olink_long_df, subset_NMR_long_df], ignore_index=True)
merged_df = pd.concat([subset_clinical_df, result_Olink_df, result_NMR_df], ignore_index=True)
print(merged_df)

            node1                  node2  relationship
0         1002133                   I802      1.000000
1         1002133                  M1997      1.000000
2         1002133                   M201      1.000000
3         1002133                   M798      1.000000
4         1002133                   D120      1.000000
...           ...                    ...           ...
15789368  6022412  participant.p23507_i0      0.092613
15789369  6022412  participant.p23506_i0      0.086871
15789370  6022412  participant.p23491_i0      0.085022
15789371  6022412  participant.p23472_i0      0.085019
15789372  6022412  participant.p23522_i0      0.085007

[15789373 rows x 3 columns]


In [54]:
# 保存为以制表符分隔的文本文件
file_path = '/home/llma/wzy/comorbidity/Data/merged_df_long.txt'
merged_df.to_csv(file_path, sep='\t', na_rep='nan', index=False)

In [55]:
merged_df = pd.read_csv('/home/llma/wzy/comorbidity/Data/merged_df_long.txt', sep='\t')

merged_df.head(2)

,node1,node2,relationship
0,1002133,I802,1.0
1,1002133,M1997,1.0


In [56]:
# 随机抽取 50000000 行数据
sampled_df = merged_df.sample(n=100000, random_state=42)

print(sampled_df)

            node1                      node2  relationship
10890217  1509466      participant.p23580_i0      20.27500
8972900   5308238    olink_instance_0.itgbl1       0.88600
14002659  4380370      participant.p23471_i0       3.48600
74229     3351479                       H185       1.00000
6305790   3905089  olink_instance_0.fam171a2       5.24180
...           ...                        ...           ...
1265761   1241521      olink_instance_0.tshb       1.90755
8766054   5199637     olink_instance_0.cdh17       1.13260
5134481   3282256      olink_instance_0.cd69       0.57720
13483563  3903901      participant.p23511_i0       0.09834
11273352  1850949      participant.p23410_i0       0.17939

[100000 rows x 3 columns]


In [57]:
# 保存为以制表符分隔的文本文件
file_path = '/home/llma/wzy/comorbidity/Data/sampled_df_long.txt'
sampled_df.to_csv(file_path, sep='\t', na_rep='nan', index=False)

## prapare data for GPT embedding

In [66]:
len(merged_df['node1'].unique())

47626

In [67]:
type(merged_df['node1'].unique())

numpy.ndarray

In [68]:
merged_df['node1'].unique()

array([1002133, 1002201, 1002534, ..., 6022081, 6022213, 6022247])

In [69]:
merged_df['node1']

0           1002133
1           1002133
2           1002133
3           1002133
4           1002133
             ...   
15789368    6022412
15789369    6022412
15789370    6022412
15789371    6022412
15789372    6022412
Name: node1, Length: 15789373, dtype: int64

In [72]:
import numpy as np
np.savetxt('/home/llma/wzy/comorbidity/Data/patient_id_select.txt', merged_df['node1'].unique(), fmt='%d')

In [73]:
import numpy as np
X = np.load('/home/llma/wzy/comorbidity/Data/UKB_node_feature.npy')

In [79]:
X.shape

(59341, 512)

In [4]:
from embedding import load_graph_network
from train import fit_transform
import torch
import numpy as np
from scipy.sparse import identity, csr_matrix

/home/llma/anaconda3/envs/GAT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
relationship_file = '/home/llma/wzy/comorbidity/Data/merged_df_long.txt'
node_feature_file = '/home/llma/wzy/comorbidity/Data/UKB_node_feature.npy'
features, adj = load_graph_network(relationship_file, node_feature_file)

In [ ]:
type(features)

In [ ]:
features.shape

In [ ]:
type(adj)

In [ ]:
adj.shape

In [86]:
edge_index = torch.LongTensor(np.column_stack(adj.nonzero())).T

In [87]:
type(edge_index)

torch.Tensor

In [88]:
edge_index.shape

torch.Size([2, 31578678])

In [89]:
edge_index

tensor([[    0,     0,     0,  ..., 59340, 59340, 59340],
        [48395, 50305, 52968,  ..., 47621, 47622, 47625]])

In [91]:
features = torch.FloatTensor(features)

In [92]:
type(features)

torch.Tensor

In [93]:
features

tensor([[ 7.8593e-02,  3.7167e-02, -2.5264e-05,  ...,  1.0927e-01,
          4.2039e-02, -4.3163e-02],
        [-3.1614e-02,  3.3373e-02, -4.1725e-02,  ...,  6.1832e-02,
          3.3184e-02, -7.6527e-02],
        [ 6.3645e-02, -6.6597e-02, -1.7813e-02,  ...,  8.3573e-02,
          1.1757e-01,  7.7124e-02],
        ...,
        [ 4.8801e-02, -1.5700e-02, -8.0005e-02,  ...,  1.8323e-01,
         -4.8555e-02, -8.9389e-02],
        [ 4.1061e-02, -3.6173e-03, -4.3467e-02,  ...,  1.9806e-01,
         -2.3962e-02, -9.1589e-02],
        [ 4.6492e-02, -1.6612e-02, -4.4272e-02,  ...,  1.7582e-01,
         -2.1368e-02, -9.3193e-02]])

In [1]:
import numpy as np

In [2]:
# 读取 .npy 文件
UKB_node_feature = np.load("/home/llma/wzy/comorbidity/Data/UKB_node_feature.npy")  # 替换为你的文件路径

In [3]:
# 查看数据
print(UKB_node_feature.shape)  # 查看数组形状
print(UKB_node_feature.dtype)  # 查看数据类型
print(UKB_node_feature)        # 打印数组内容

(59341, 512)
float32
[[ 7.8592859e-02  3.7166961e-02 -2.5264493e-05 ...  1.0927329e-01
   4.2039458e-02 -4.3163471e-02]
 [-3.1613640e-02  3.3372585e-02 -4.1725088e-02 ...  6.1832469e-02
   3.3184074e-02 -7.6527424e-02]
 [ 6.3645303e-02 -6.6597082e-02 -1.7813137e-02 ...  8.3572939e-02
   1.1757230e-01  7.7123806e-02]
 ...
 [ 4.8800856e-02 -1.5699746e-02 -8.0004506e-02 ...  1.8323483e-01
  -4.8554923e-02 -8.9388750e-02]
 [ 4.1061401e-02 -3.6173388e-03 -4.3467034e-02 ...  1.9806254e-01
  -2.3962140e-02 -9.1588542e-02]
 [ 4.6492204e-02 -1.6611662e-02 -4.4271532e-02 ...  1.7581843e-01
  -2.1368263e-02 -9.3193032e-02]]


## get the order of nodes

In [39]:
import numpy as np
import os
from scipy.sparse import csr_matrix
from utils import N2V
import pandas as pd
from fastnode2vec import Graph, Node2Vec 

In [40]:
adj_path = '/home/llma/wzy/comorbidity/Data/merged_df_long_subset.txt'

In [41]:
X, A, Y = [], None, []
n_node = 0

# Acquire Edges
edge_list = []
node_list = []
node_type = {}
relationship_list = []

with open(adj_path, 'rt', encoding='utf-8') as f:
    next(f)
    for line in f.readlines():
        node1, node2, relationship, *_ = line.strip().split('\t')
        edge_list.append((node1, node2))
        node_list.extend([node1, node2])
        relationship_list.append(relationship)

node_map = {item: i for i, item in enumerate(sorted(list(set(node_list))))}
n_node = len(node_map)

# 使用稀疏矩阵存储邻接关系
row = []
col = []
data = []
for i, (node1, node2) in enumerate(edge_list):
    row.append(node_map[node1])
    col.append(node_map[node2])
    data.append(relationship_list[i])  # 使用实际的关系值
    row.append(node_map[node2])
    col.append(node_map[node1])
    data.append(relationship_list[i])  # 使用实际的关系值
A = csr_matrix((data, (row, col)), shape=(n_node, n_node), dtype=np.float32)

adj_matrix = pd.read_csv(adj_path, sep='\t')
# 构建图所需的边列表
edges = [(row['node1'], row['node2'], row['relationship']) for _, row in adj_matrix.iterrows()]

# 构建图
graph = Graph(edges, directed=False, weighted=True)

In [42]:
graph.node_names

array(['1000021', 'ccar2', 'paep', ...,
       'Concentration of Very Large HDL Particles',
       'Concentration of IDL Particles',
       'Concentration of Medium LDL Particles'], dtype='<U86')

In [43]:
graph

In [44]:
# 保存图对象
with open('/home/llma/wzy/comorbidity/Data/graph.pkl', 'wb') as f:
    pickle.dump(graph, f)

In [45]:
import pickle
import numpy as np

In [46]:
print(np.__version__)

1.26.4


In [47]:
with open('/home/llma/wzy/comorbidity/Data/gpt_protein_embeddings_dict.pkl', 'rb') as f:
    protein_embeddings_dict = pickle.load(f)

In [48]:
with open('/home/llma/wzy/comorbidity/Data/gpt_embeddings_dict.pkl', 'rb') as f:
    embeddings_dict = pickle.load(f)

In [49]:
with open('/home/llma/wzy/comorbidity/Data/metabolite_embeddings_dict.pkl', 'rb') as f:
    metabolite_embeddings_dict = pickle.load(f)

In [50]:
# 创建新字典，保留原始字典不变
merged_dict = {**protein_embeddings_dict, **embeddings_dict, **metabolite_embeddings_dict}

In [51]:
# 提取所有键并保持顺序（Python 3.7+ 字典保持插入顺序）
keys = list(graph.node_names)

# 将字典值转换为NumPy矩阵（每行一个向量）
matrix = np.array([merged_dict[key] for key in keys])

# 检查矩阵形状
print("矩阵形状:", matrix.shape)  # 输出: (47626, 向量维度)

# 示例：查看前3行
print("前3行:\n", matrix[:3])

矩阵形状: (50792, 1536)
前3行:
 [[-0.02587547  0.00406624 -0.00236452 ...  0.00774806 -0.01663612
  -0.0286592 ]
 [-0.0407948  -0.00324398 -0.00015634 ... -0.0127301  -0.0239985
   0.00535847]
 [-0.0363668  -0.01535156 -0.00880039 ... -0.01902346 -0.00901718
  -0.02430776]]


In [52]:
np.save('/home/llma/wzy/comorbidity/Data/UKB_node_feature_gpt.npy', matrix)

## split train data, validation data, test data

In [57]:
merged_df_long = pd.read_csv('/home/llma/wzy/comorbidity/Data/merged_df_long_subset.txt', sep='\t')

In [58]:
merged_df_long.head(2)

,node1,node2,relationship
0,1000021,ccar2,1
1,1000021,paep,1


In [60]:
def split_data_by_patient_id(data_df, patient_id_col='node1', ratios=[7, 2, 1], random_seed=42):
    """
    根据病人ID将数据框按指定比例划分为多个子集，确保各子集间病人ID无重叠
    
    参数:
    data_df: pandas.DataFrame - 输入的数据框
    patient_id_col: str - 病人ID所在的列名，默认为'node1'
    ratios: list - 划分比例，默认为[7, 2, 1]
    random_seed: int - 随机数种子，确保结果可重现
    
    返回:
    tuple - 划分后的多个数据框
    """
    # 设置随机种子
    np.random.seed(random_seed)
    
    # 获取唯一的病人ID
    unique_patient_ids = data_df[patient_id_col].unique()
    total_patients = len(unique_patient_ids)
    
    # 计算各部分应占的病人数量
    ratio_sum = sum(ratios)
    parts = []
    remaining = total_patients
    
    for i, ratio in enumerate(ratios[:-1]):
        # 计算当前部分的病人数量，四舍五入
        current_size = round(total_patients * ratio / ratio_sum)
        # 确保不会超出总人数
        current_size = min(current_size, remaining)
        parts.append(current_size)
        remaining -= current_size
    
    # 最后一部分包含剩余的所有病人
    parts.append(remaining)
    
    # 随机打乱病人ID
    shuffled_ids = np.random.permutation(unique_patient_ids)
    
    # 划分病人ID
    split_points = [0]
    for size in parts:
        split_points.append(split_points[-1] + size)
    
    patient_groups = [
        shuffled_ids[split_points[i]:split_points[i+1]]
        for i in range(len(parts))
    ]
    
    # 根据病人ID划分数据
    data_groups = [
        data_df[data_df[patient_id_col].isin(patients)]
        for patients in patient_groups
    ]
    
    return tuple(data_groups)

In [61]:
train_data, val_data, test_data = split_data_by_patient_id(
    data_df=merged_df_long,          # 你的原始数据框
    patient_id_col='node1',  # 病人ID列
    ratios=[7, 2, 1],   # 对应测试集:验证集:训练集
    random_seed=42      # 可设置随机种子确保结果可重现
)

In [62]:
# 查看各部分大小
print(f"训练集: {len(train_data)} 行")
print(f"验证集: {len(val_data)} 行")
print(f"测试集: {len(test_data)} 行")

训练集: 10493322 行
验证集: 2999875 行
测试集: 1490800 行


In [65]:
train_data.to_csv('/home/llma/wzy/comorbidity/Data/train_data.txt', sep='\t', na_rep='nan', index=False)
val_data.to_csv('/home/llma/wzy/comorbidity/Data/val_data.txt', sep='\t', na_rep='nan', index=False)
test_data.to_csv('/home/llma/wzy/comorbidity/Data/test_data.txt', sep='\t', na_rep='nan', index=False)

In [64]:
train_data.head(2)

,node1,node2,relationship
0,1000021,ccar2,1
1,1000021,paep,1


In [68]:
def prepareInputData(adj_path,
                     graph_path_save,
                     protein_embeddings_path,
                     gpt_embeddings_path,
                     metabolite_embeddings_path,
                     UKB_node_feature_save
                     ):
    X, A, Y = [], None, []
    n_node = 0
    
    # Acquire Edges
    edge_list = []
    node_list = []
    node_type = {}
    relationship_list = []
    
    with open(adj_path, 'rt', encoding='utf-8') as f:
        next(f)
        for line in f.readlines():
            node1, node2, relationship, *_ = line.strip().split('\t')
            edge_list.append((node1, node2))
            node_list.extend([node1, node2])
            relationship_list.append(relationship)
    
    node_map = {item: i for i, item in enumerate(sorted(list(set(node_list))))}
    n_node = len(node_map)
    
    # 使用稀疏矩阵存储邻接关系
    row = []
    col = []
    data = []
    for i, (node1, node2) in enumerate(edge_list):
        row.append(node_map[node1])
        col.append(node_map[node2])
        data.append(relationship_list[i])  # 使用实际的关系值
        row.append(node_map[node2])
        col.append(node_map[node1])
        data.append(relationship_list[i])  # 使用实际的关系值
    A = csr_matrix((data, (row, col)), shape=(n_node, n_node), dtype=np.float32)
    
    adj_matrix = pd.read_csv(adj_path, sep='\t')
    # 构建图所需的边列表
    edges = [(row['node1'], row['node2'], row['relationship']) for _, row in adj_matrix.iterrows()]
    
    # 构建图
    graph = Graph(edges, directed=False, weighted=True)
    # 保存图对象
    with open(graph_path_save, 'wb') as f:
        pickle.dump(graph, f)

    with open(protein_embeddings_path, 'rb') as f:
        protein_embeddings_dict = pickle.load(f)

    with open(gpt_embeddings_path, 'rb') as f:
        embeddings_dict = pickle.load(f)
    
    with open(metabolite_embeddings_path, 'rb') as f:
        metabolite_embeddings_dict = pickle.load(f)
    
    # 创建新字典，保留原始字典不变
    merged_dict = {**protein_embeddings_dict, **embeddings_dict, **metabolite_embeddings_dict}
    # 提取所有键并保持顺序（Python 3.7+ 字典保持插入顺序）
    keys = list(graph.node_names)
    
    # 将字典值转换为NumPy矩阵（每行一个向量）
    matrix = np.array([merged_dict[key] for key in keys])
    # 检查矩阵形状
    print("矩阵形状:", matrix.shape)  # 输出: (47626, 向量维度)
    np.save(UKB_node_feature_save, matrix)

In [69]:
prepareInputData(adj_path='/home/llma/wzy/comorbidity/Data/train_data.txt',
                 graph_path_save='/home/llma/wzy/comorbidity/Data/train_graph.pkl',
                 protein_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_protein_embeddings_dict.pkl',
                 gpt_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_embeddings_dict.pkl',
                 metabolite_embeddings_path='/home/llma/wzy/comorbidity/Data/metabolite_embeddings_dict.pkl',
                 UKB_node_feature_save='/home/llma/wzy/comorbidity/Data/UKB_node_feature_gpt_train.npy'
                     )

矩阵形状: (36504, 1536)


In [71]:
prepareInputData(adj_path='/home/llma/wzy/comorbidity/Data/val_data.txt',
                 graph_path_save='/home/llma/wzy/comorbidity/Data/val_graph.pkl',
                 protein_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_protein_embeddings_dict.pkl',
                 gpt_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_embeddings_dict.pkl',
                 metabolite_embeddings_path='/home/llma/wzy/comorbidity/Data/metabolite_embeddings_dict.pkl',
                 UKB_node_feature_save='/home/llma/wzy/comorbidity/Data/UKB_node_feature_gpt_val.npy'
                     )

矩阵形状: (12681, 1536)


In [70]:
prepareInputData(adj_path='/home/llma/wzy/comorbidity/Data/test_data.txt',
                 graph_path_save='/home/llma/wzy/comorbidity/Data/test_graph.pkl',
                 protein_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_protein_embeddings_dict.pkl',
                 gpt_embeddings_path='/home/llma/wzy/comorbidity/Data/gpt_embeddings_dict.pkl',
                 metabolite_embeddings_path='/home/llma/wzy/comorbidity/Data/metabolite_embeddings_dict.pkl',
                 UKB_node_feature_save='/home/llma/wzy/comorbidity/Data/UKB_node_feature_gpt_test.npy'
                     )

矩阵形状: (7919, 1536)
